In [2]:
import glob
import json
import csv
import pandas as pd

In [5]:
## Extract Json to CSV

fields = ["filename", "class", "fill","x","y","width","height"]
classes = ["glass","cup","plate"]


def json_to_csv(split=False):
    df = []
    output_name = "csv_converted.csv"
    for fn in sorted(glob.glob("dataset/TDB_M/*.json")):
        f = open(fn)
        json_obj = json.load(f)
        for key in json_obj:
            dict_row = {"filename":json_obj[key]["filename"],
                        "class":"null",
                        "fill":"null",
                        "x":"null",
                        "y":"null",
                        "width":"null",
                        "height":"null"}

            if len(json_obj[key]["regions"]) == 0:
                df.append(dict_row)

            else:
                temp_c = []
                temp_fill = []
                temp_x =[]
                temp_y = []
                temp_width= []
                temp_height = []
                for r in json_obj[key]["regions"]:
                    if "Type" in json_obj[key]["regions"][r]["region_attributes"]:
                        c = json_obj[key]["regions"][r]["region_attributes"]["Type"].lower().strip()
                        if "Fill" in json_obj[key]["regions"][r]["region_attributes"]:
                            fill = json_obj[key]["regions"][r]["region_attributes"]["Fill"].lower().strip()
                            if fill in classes:
                                fill = 30
                    elif "type" in json_obj[key]["regions"][r]["region_attributes"]:
                        c = json_obj[key]["regions"][r]["region_attributes"]["type"].lower().strip()
                        if "fill" in json_obj[key]["regions"][r]["region_attributes"]:
                            fill = json_obj[key]["regions"][r]["region_attributes"]["fill"].lower().strip()
                            if fill in classes:
                                fill = 30
                                
                    temp_c.append(c)
                    temp_fill.append(fill)
                    
                    sa = json_obj[key]["regions"][r]["shape_attributes"]
                    x = sa["x"]
                    y = sa["y"]
                    width = sa["width"]
                    height = sa["height"]
                    
                    temp_x.append(x)
                    temp_y.append(y)
                    temp_width.append(width)
                    temp_height.append(height)
                    
                    
                if (split):
                    dict_row["class"] = temp_c
                    dict_row["fill"] = temp_fill
                    dict_row["x"] = temp_x
                    dict_row["y"] = temp_y
                    dict_row["width"] = temp_width
                    dict_row["height"] = temp_height
                else:
                    dict_row["class"] = ";".join(temp_c)
                    dict_row["fill"] = ";".join(map(str, temp_fill))
                    dict_row["x"] = ";".join(map(str, temp_x))
                    dict_row["y"] = ";".join(map(str, temp_y))
                    dict_row["width"] = ";".join(map(str, temp_width))
                    dict_row["height"] = ";".join(map(str, temp_height))
                
                df.append(dict_row)
    
    df = pd.DataFrame(df)
    df = df[fields]
    df = df.drop_duplicates(subset=['filename'], keep='last')
    print(df)
    
    if (split):
        df = df.set_index(['filename']).apply(pd.Series.explode).reset_index()
        output_name = output_name[:-4]+"_splitted.csv"
        print(output_name)
        print(df)
    
    df.to_csv(output_name,index=False,header=True, sep=",")
    
## use split = True if you want splitted output
## or False if you want merged output
json_to_csv(True)

          filename                                class                  fill  \
0     0001.jpg.png                                 null                  null   
1        00002.jpg                                 null                  null   
2        00003.jpg                                 null                  null   
3        00004.jpg                                 null                  null   
4        00005.jpg                                 null                  null   
...            ...                                  ...                   ...   
2051     01996.jpg  [glass, glass, glass, glass, glass]  [30, 30, 30, 30, 30]   
2052     01997.jpg  [glass, glass, glass, glass, glass]  [30, 30, 30, 30, 30]   
2053     01998.jpg  [glass, glass, glass, glass, glass]  [30, 30, 30, 30, 30]   
2054     01999.jpg  [glass, glass, glass, glass, glass]  [30, 30, 30, 30, 30]   
2055     02000.jpg  [glass, glass, glass, glass, glass]  [30, 30, 30, 30, 30]   

                           

In [9]:
## Clean Data
import pandas as pd

df = pd.read_csv("csv_converted_splitted.csv")
print("raw shape", df.shape)
print("raw columns", df.columns)

# delete plates (reduce the size from 6477 to 6167)
cups_glasses = df[df["class"] != "plate"]

print("we still have nans", cups_glasses["class"].unique())
cups_glasses = cups_glasses.dropna()
print("only cups and glasses", cups_glasses.shape)
# size reduced from 6167 to 6117

acceptable_fills = ["0", "30", "100"]
cups_glasses_reduced = cups_glasses[cups_glasses["fill"].isin(acceptable_fills)]
print("only allowed fills", cups_glasses_reduced.shape)
# size reduced from 6117 to 5993

# write to a csv
cups_glasses_reduced.to_csv("reduced_cups_glasses.csv", index=False)

raw shape (6477, 7)
raw columns Index(['filename', 'class', 'fill', 'x', 'y', 'width', 'height'], dtype='object')
we still have nans [nan 'glass' 'cup']
only cups and glasses (6117, 7)
only allowed fills (5993, 7)


In [3]:
# Split the dataset to Train(80%) and Test(20%)
# df = pd.read_csv("balanced_dataset.csv")
df = pd.read_csv("reduced_cups_glasses.csv")
print(df.shape)

# Creating Test dataset
## Glass 0%
test = df[(df["fill"] == 0) & (df["class"] == "glass")].sample(frac=0.2)
## Glass 30%
test = test.append(df[(df["fill"] == 30) & (df["class"] == "glass")].sample(frac=0.2))
## Glass 100%
test = test.append(df[(df["fill"] == 100) & (df["class"] == "glass")].sample(frac=0.2))
## Cup 0%
test = test.append(df[(df["fill"] == 0) & (df["class"] == "cup")].sample(frac=0.2))
## Cup 30%
test = test.append(df[(df["fill"] == 30) & (df["class"] == "cup")].sample(frac=0.2))
## Cup 100%
test = test.append(df[(df["fill"] == 100) & (df["class"] == "cup")].sample(frac=0.2))

# Create train dataset by excluding the test dataset
train = pd.concat([df, test, test]).drop_duplicates(keep=False)

print(test)
print(test.shape)
print(train)
print(train.shape)

train.to_csv("train_dataset.csv", index = False)
test.to_csv("test_dataset.csv", index = False)


(5993, 7)
       filename  class  fill       x      y  width  height
68    00099.jpg  glass     0   692.0  782.0  203.0   246.0
2041  00767.jpg  glass     0  1128.0  592.0  197.0   231.0
2493  00885.jpg  glass     0  1136.0  658.0  182.0   212.0
4009  01348.jpg  glass     0   357.0  547.0  265.0   229.0
4180  01420.jpg  glass     0   920.0  599.0  167.0   218.0
...         ...    ...   ...     ...    ...    ...     ...
1429  00595.jpg    cup   100  1383.0  107.0  300.0   326.0
1268  00556.jpg    cup   100  1002.0  786.0  228.0   235.0
886   00465.jpg    cup   100  1405.0  841.0  271.0   194.0
924   00476.jpg    cup   100   992.0   98.0  278.0   236.0
845   00455.jpg    cup   100   630.0  272.0  228.0   282.0

[1199 rows x 7 columns]
(1199, 7)
       filename  class  fill       x      y  width  height
0     00051.jpg  glass     0   478.0  644.0  262.0   290.0
1     00052.jpg  glass     0   609.0  423.0  238.0   223.0
2     00052.jpg  glass     0   786.0  629.0  250.0   390.0
3     00053

In [4]:
import sys
sys.path.insert(0, 'D:\\Project\\AiTalents\\models\\')
sys.path.insert(0, 'D:\\Project\\AiTalents\\models\\research')
sys.path.insert(0, 'D:\\Project\\AiTalents\\models\\research\\object_detection')
print(sys.path)

['D:\\Project\\AiTalents\\models\\research\\object_detection', 'D:\\Project\\AiTalents\\models\\research', 'D:\\Project\\AiTalents\\models\\', 'D:\\Project\\AiTalents', 'D:\\Project\\AiTalents\\models', 'D:\\Project\\AiTalents\\models\\research', 'D:\\Project\\AiTalents\\models\\research\\object_detection', 'D:\\Project\\AiTalents\\models\\research\\slim', 'D:\\Project\\AiTalents', 'C:\\Users\\User\\anaconda3\\envs\\tensorflow\\python38.zip', 'C:\\Users\\User\\anaconda3\\envs\\tensorflow\\DLLs', 'C:\\Users\\User\\anaconda3\\envs\\tensorflow\\lib', 'C:\\Users\\User\\anaconda3\\envs\\tensorflow', '', 'C:\\Users\\User\\AppData\\Roaming\\Python\\Python38\\site-packages', 'C:\\Users\\User\\anaconda3\\envs\\tensorflow\\lib\\site-packages', 'C:\\Users\\User\\anaconda3\\envs\\tensorflow\\lib\\site-packages\\win32', 'C:\\Users\\User\\anaconda3\\envs\\tensorflow\\lib\\site-packages\\win32\\lib', 'C:\\Users\\User\\anaconda3\\envs\\tensorflow\\lib\\site-packages\\Pythonwin', 'C:\\Users\\User\\anac

In [5]:
## Functions to converting dataset to TFRecord
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd

import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple

def class_text_to_int(row_label):
    cls = {"glass_0":0, "glass_30":1, "glass_100":2, "cup_0":3, "cup_30":4, "cup_100":5}
    return cls[row_label]

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['x'] / width)
        xmaxs.append((row['x']+row['width']) / width)
        ymins.append(row['y'] / height)
        ymaxs.append((row['y']+row['height']) / height)
#         print(row["class"])
#         print(row["fill"])
        cls_fill = str(row['class'])+"_"+str(row['fill'])
        print(cls_fill)
        classes_text.append(cls_fill.encode('utf8'))
        classes.append(class_text_to_int(cls_fill))
    
    tf_example = tf.train.Example(features=tf.train.Features(feature={
    'image/height': dataset_util.int64_feature(height),
    'image/width': dataset_util.int64_feature(width),
    'image/filename': dataset_util.bytes_feature(filename),
    'image/source_id': dataset_util.bytes_feature(filename),
    'image/encoded': dataset_util.bytes_feature(encoded_jpg),
    'image/format': dataset_util.bytes_feature(image_format),
    'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
    'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
    'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
    'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
    'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
    'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def convert_df_to_tfrecord(images_path,input_df, output_path):
    with tf.io.TFRecordWriter(output_path) as writer:
        path = os.path.join(os.getcwd(), images_path)
        grouped = split(input_df, 'filename')
        for group in grouped:
            tf_example = create_tf_example(group, path)
            writer.write(tf_example.SerializeToString())

    print('Successfully created the TFRecords: {}'.format(output_path))

In [9]:
## Resize image
import cv2
from pathlib import Path
import numpy as np

def resize_image(size, input_path, output_path, x, y, width, height):
    img = cv2.imread(input_path)
    
    y_ = img.shape[0]
    x_ = img.shape[1]
    print(x_,y_)

    x_scale = size / x_
    y_scale = size / y_
    print(x_scale, y_scale)
    img = cv2.resize(img, (size,size), interpolation = cv2.INTER_AREA)
    print(img.shape)
    img = np.array(img);

    x = int(np.round(x * x_scale))
    y = int(np.round(y * y_scale))
    xmax = int(np.round(x+width * x_scale))
    ymax = int(np.round(y+height * y_scale))
    
#     img = cv2.rectangle(img, (int(x),int(y)), (int(xmax),int(ymax)), (255,0,255), 1)

    cv2.imwrite(output_path, img)
    return x,y,xmax,ymax

def split_dataset(input_path, output_path):
    if(not os.path.exists(output_path)):
        img = cv2.imread(input_path)
        cv2.imwrite(output_path, img)
    
df_train = pd.read_csv("train_dataset.csv")
df_test = pd.read_csv("test_dataset.csv")

dataset_path = "dataset/TDB_M/"
# output_path = "dataset/TDB_M_320/"
output_path = "dataset/TDB_M_splitted/"
output_train_path = output_path+"Train/"
output_test_path = output_path+"Test/"

path = Path(output_train_path)
path.mkdir(parents=True, exist_ok=True)

path = Path(output_test_path)
path.mkdir(parents=True, exist_ok=True)

## Resize The datasets to 320x320
for index, row in df_train.iterrows():
    print(row['filename'])
#     resize_image(320,dataset_path+row['filename'], output_train_path+row['filename'],row['x'],row['y'],row['width'],row['height'])
    split_dataset(dataset_path+row['filename'], output_train_path+row['filename'])

for index, row in df_test.iterrows():
    print(row['filename'])
#     resize_image(320,dataset_path+row['filename'], output_test_path+row['filename'],row['x'],row['y'],row['width'],row['height'])
    split_dataset(dataset_path+row['filename'], output_test_path+row['filename'])


00051.jpg
00052.jpg
00052.jpg
00053.jpg
00054.jpg
00055.jpg
00055.jpg
00056.jpg
00057.jpg
00058.jpg
00062.jpg
00063.jpg
00068.jpg
00069.jpg
00070.jpg
00071.jpg
00072.jpg
00072.jpg
00074.jpg
00075.jpg
00076.jpg
00077.jpg
00077.jpg
00077.jpg
00078.jpg
00078.jpg
00078.jpg
00079.jpg
00079.jpg
00080.jpg
00080.jpg
00081.jpg
00081.jpg
00082.jpg
00083.jpg
00084.jpg
00086.jpg
00088.jpg
00089.jpg
00089.jpg
00090.jpg
00091.jpg
00092.jpg
00094.jpg
00094.jpg
00095.jpg
00095.jpg
00095.jpg
00096.jpg
00097.jpg
00098.jpg
00098.jpg
00099.jpg
00100.jpg
00100.jpg
00102.jpg
00103.jpg
00103.jpg
00104.jpg
00105.jpg
00105.jpg
00106.jpg
00107.jpg
00108.jpg
00110.jpg
00111.jpg
00114.jpg
00115.jpg
00116.jpg
00117.jpg
00118.jpg
00118.jpg
00119.jpg
00120.jpg
00121.jpg
00121.jpg
00122.jpg
00123.jpg
00124.jpg
00124.jpg
00125.jpg
00126.jpg
00128.jpg
00129.jpg
00131.jpg
00132.jpg
00134.jpg
00134.jpg
00135.jpg
00136.jpg
00136.jpg
00137.jpg
00138.jpg
00139.jpg
00139.jpg
00140.jpg
00141.jpg
00142.jpg
00143.jpg
00144.jpg


00504.jpg
00505.jpg
00505.jpg
00505.jpg
00506.jpg
00507.jpg
00507.jpg
00507.jpg
00507.jpg
00507.jpg
00508.jpg
00508.jpg
00508.jpg
00508.jpg
00509.jpg
00509.jpg
00509.jpg
00510.jpg
00510.jpg
00510.jpg
00510.jpg
00510.jpg
00511.jpg
00511.jpg
00512.jpg
00512.jpg
00512.jpg
00512.jpg
00512.jpg
00513.jpg
00513.jpg
00514.jpg
00514.jpg
00514.jpg
00515.jpg
00515.jpg
00516.jpg
00516.jpg
00516.jpg
00516.jpg
00516.jpg
00517.jpg
00517.jpg
00517.jpg
00518.jpg
00518.jpg
00518.jpg
00518.jpg
00518.jpg
00519.jpg
00519.jpg
00519.jpg
00519.jpg
00520.jpg
00520.jpg
00520.jpg
00520.jpg
00521.jpg
00521.jpg
00521.jpg
00521.jpg
00521.jpg
00522.jpg
00522.jpg
00522.jpg
00523.jpg
00523.jpg
00523.jpg
00523.jpg
00524.jpg
00525.jpg
00525.jpg
00526.jpg
00526.jpg
00526.jpg
00527.jpg
00527.jpg
00527.jpg
00528.jpg
00528.jpg
00528.jpg
00528.jpg
00529.jpg
00529.jpg
00530.jpg
00530.jpg
00530.jpg
00531.jpg
00531.jpg
00531.jpg
00531.jpg
00532.jpg
00532.jpg
00532.jpg
00533.jpg
00533.jpg
00533.jpg
00534.jpg
00534.jpg
00534.jpg


00768.jpg
00769.jpg
00769.jpg
00769.jpg
00769.jpg
00770.jpg
00770.jpg
00771.jpg
00771.jpg
00771.jpg
00771.jpg
00771.jpg
00772.jpg
00772.jpg
00772.jpg
00772.jpg
00772.jpg
00773.jpg
00773.jpg
00773.jpg
00773.jpg
00773.jpg
00774.jpg
00774.jpg
00774.jpg
00774.jpg
00775.jpg
00775.jpg
00776.jpg
00776.jpg
00776.jpg
00776.jpg
00777.jpg
00777.jpg
00777.jpg
00777.jpg
00778.jpg
00778.jpg
00778.jpg
00779.jpg
00779.jpg
00780.jpg
00780.jpg
00780.jpg
00780.jpg
00780.jpg
00781.jpg
00781.jpg
00781.jpg
00782.jpg
00782.jpg
00782.jpg
00782.jpg
00782.jpg
00783.jpg
00783.jpg
00783.jpg
00783.jpg
00783.jpg
00784.jpg
00784.jpg
00784.jpg
00784.jpg
00784.jpg
00785.jpg
00785.jpg
00785.jpg
00785.jpg
00785.jpg
00786.jpg
00786.jpg
00786.jpg
00786.jpg
00787.jpg
00787.jpg
00787.jpg
00788.jpg
00788.jpg
00788.jpg
00788.jpg
00788.jpg
00789.jpg
00789.jpg
00789.jpg
00790.jpg
00790.jpg
00790.jpg
00791.jpg
00791.jpg
00791.jpg
00791.jpg
00792.jpg
00793.jpg
00793.jpg
00793.jpg
00793.jpg
00794.jpg
00794.jpg
00794.jpg
00795.jpg


01075.jpg
01075.jpg
01075.jpg
01076.jpg
01076.jpg
01076.jpg
01076.jpg
01077.jpg
01077.jpg
01077.jpg
01077.jpg
01078.jpg
01078.jpg
01078.jpg
01078.jpg
01079.jpg
01079.jpg
01079.jpg
01079.jpg
01080.jpg
01080.jpg
01080.jpg
01080.jpg
01081.jpg
01081.jpg
01081.jpg
01081.jpg
01082.jpg
01082.jpg
01082.jpg
01082.jpg
01083.jpg
01083.jpg
01083.jpg
01084.jpg
01084.jpg
01084.jpg
01084.jpg
01085.jpg
01085.jpg
01085.jpg
01086.jpg
01086.jpg
01086.jpg
01087.jpg
01087.jpg
01088.jpg
01088.jpg
01088.jpg
01088.jpg
01089.jpg
01089.jpg
01089.jpg
01089.jpg
01090.jpg
01090.jpg
01090.jpg
01091.jpg
01091.jpg
01091.jpg
01092.jpg
01092.jpg
01092.jpg
01093.jpg
01093.jpg
01094.jpg
01094.jpg
01095.jpg
01095.jpg
01095.jpg
01096.jpg
01096.jpg
01096.jpg
01096.jpg
01097.jpg
01097.jpg
01098.jpg
01098.jpg
01100.jpg
01100.jpg
01100.jpg
01101.jpg
01101.jpg
01101.jpg
01101.jpg
01102.jpg
01102.jpg
01102.jpg
01103.jpg
01103.jpg
01104.jpg
01104.jpg
01105.jpg
01105.jpg
01105.jpg
01106.jpg
01106.jpg
01107.jpg
01107.jpg
01107.jpg


01399.jpg
01399.jpg
01401.jpg
01401.jpg
01402.jpg
01402.jpg
01402.jpg
01403.jpg
01403.jpg
01404.jpg
01404.jpg
01404.jpg
01405.jpg
01405.jpg
01406.jpg
01406.jpg
01407.jpg
01407.jpg
01408.jpg
01409.jpg
01409.jpg
01410.jpg
01410.jpg
01410.jpg
01411.jpg
01411.jpg
01411.jpg
01412.jpg
01412.jpg
01412.jpg
01412.jpg
01413.jpg
01413.jpg
01413.jpg
01413.jpg
01414.jpg
01415.jpg
01415.jpg
01415.jpg
01415.jpg
01416.jpg
01416.jpg
01416.jpg
01417.jpg
01417.jpg
01417.jpg
01417.jpg
01418.jpg
01418.jpg
01418.jpg
01419.jpg
01420.jpg
01420.jpg
01420.jpg
01421.jpg
01421.jpg
01421.jpg
01421.jpg
01422.jpg
01422.jpg
01423.jpg
01423.jpg
01424.jpg
01424.jpg
01424.jpg
01425.jpg
01426.jpg
01426.jpg
01426.jpg
01427.jpg
01427.jpg
01428.jpg
01428.jpg
01428.jpg
01429.jpg
01429.jpg
01430.jpg
01430.jpg
01431.jpg
01431.jpg
01431.jpg
01432.jpg
01432.jpg
01433.jpg
01433.jpg
01433.jpg
01434.jpg
01435.jpg
01435.jpg
01435.jpg
01436.jpg
01436.jpg
01437.jpg
01437.jpg
01437.jpg
01437.jpg
01438.jpg
01438.jpg
01438.jpg
01438.jpg


01788.jpg
01788.jpg
01789.jpg
01789.jpg
01789.jpg
01790.jpg
01790.jpg
01790.jpg
01791.jpg
01791.jpg
01791.jpg
01792.jpg
01792.jpg
01793.jpg
01793.jpg
01794.jpg
01795.jpg
01795.jpg
01796.jpg
01796.jpg
01796.jpg
01797.jpg
01797.jpg
01797.jpg
01798.jpg
01798.jpg
01798.jpg
01799.jpg
01799.jpg
01800.jpg
01800.jpg
01801.jpg
01802.jpg
01802.jpg
01802.jpg
01803.jpg
01803.jpg
01803.jpg
01804.jpg
01804.jpg
01806.jpg
01806.jpg
01806.jpg
01807.jpg
01807.jpg
01807.jpg
01808.jpg
01809.jpg
01809.jpg
01810.jpg
01810.jpg
01810.jpg
01811.jpg
01811.jpg
01811.jpg
01812.jpg
01812.jpg
01813.jpg
01813.jpg
01813.jpg
01814.jpg
01814.jpg
01815.jpg
01815.jpg
01816.jpg
01816.jpg
01816.jpg
01817.jpg
01817.jpg
01817.jpg
01818.jpg
01818.jpg
01818.jpg
01819.jpg
01819.jpg
01820.jpg
01820.jpg
01820.jpg
01821.jpg
01821.jpg
01821.jpg
01822.jpg
01822.jpg
01823.jpg
01823.jpg
01824.jpg
01824.jpg
01825.jpg
01825.jpg
01826.jpg
01826.jpg
01827.jpg
01827.jpg
01827.jpg
01828.jpg
01828.jpg
01828.jpg
01829.jpg
01829.jpg
01829.jpg


00789.jpg
00060.jpg
00388.jpg
01322.jpg
01504.jpg
01925.jpg
01282.jpg
01228.jpg
01610.jpg
00302.jpg
00638.jpg
01964.jpg
01057.jpg
00665.jpg
01988.jpg
01599.jpg
00821.jpg
01849.jpg
01970.jpg
00950.jpg
00937.jpg
01861.jpg
01116.jpg
01330.jpg
01046.jpg
01014.jpg
00686.jpg
01633.jpg
01013.jpg
01893.jpg
01660.jpg
01980.jpg
00958.jpg
01612.jpg
01601.jpg
00661.jpg
01132.jpg
01714.jpg
01466.jpg
01551.jpg
01550.jpg
00807.jpg
00853.jpg
01917.jpg
01761.jpg
01337.jpg
01487.jpg
01148.jpg
00849.jpg
00684.jpg
01297.jpg
01037.jpg
01998.jpg
00971.jpg
01333.jpg
01707.jpg
01922.jpg
01146.jpg
01844.jpg
01954.jpg
01756.jpg
01895.jpg
01225.jpg
01455.jpg
01590.jpg
01036.jpg
00291.jpg
01072.jpg
01933.jpg
01754.jpg
01639.jpg
01903.jpg
01781.jpg
01866.jpg
00653.jpg
01143.jpg
01973.jpg
01278.jpg
01937.jpg
01502.jpg
01242.jpg
01590.jpg
01545.jpg
01109.jpg
01238.jpg
01023.jpg
01483.jpg
01522.jpg
01155.jpg
01341.jpg
01928.jpg
01740.jpg
00640.jpg
01310.jpg
01330.jpg
01516.jpg
01938.jpg
01512.jpg
01567.jpg
01032.jpg


00112.jpg
00394.jpg
00081.jpg
00172.jpg
00382.jpg
00426.jpg
00325.jpg
00491.jpg
00154.jpg
00413.jpg
00085.jpg
00417.jpg
00500.jpg
00483.jpg
00484.jpg
00182.jpg
00513.jpg
00465.jpg
00415.jpg
00486.jpg
00487.jpg
00190.jpg
00189.jpg
00375.jpg
00467.jpg
00457.jpg
00485.jpg
00415.jpg
00123.jpg
00429.jpg
00082.jpg
00414.jpg
00486.jpg
00127.jpg
00502.jpg
00104.jpg
00462.jpg
00109.jpg
00455.jpg
00434.jpg
00465.jpg
00425.jpg
00161.jpg
00415.jpg
00122.jpg
00404.jpg
00312.jpg
00152.jpg
00498.jpg
00433.jpg
00168.jpg
00413.jpg
00431.jpg
00120.jpg
00523.jpg
00422.jpg
00312.jpg
00113.jpg
00332.jpg
00420.jpg
00322.jpg
00079.jpg
00421.jpg
00432.jpg
00509.jpg
00059.jpg
00412.jpg
00412.jpg
00064.jpg
00615.jpg
00630.jpg
00524.jpg
00615.jpg
00577.jpg
00232.jpg
00239.jpg
00610.jpg
00334.jpg
00447.jpg
00508.jpg
00620.jpg
00619.jpg
00558.jpg
00556.jpg
00364.jpg
00313.jpg
00561.jpg
00372.jpg
00530.jpg
00451.jpg
00251.jpg
00605.jpg
00557.jpg
00629.jpg
00546.jpg
00459.jpg
00470.jpg
00454.jpg
00459.jpg
00287.jpg


In [10]:
# Create TFRecord
convert_df_to_tfrecord(output_train_path,df_train, "train.tfrecord")
convert_df_to_tfrecord(output_test_path,df_test, "test.tfrecord")

glass_0
glass_0
glass_0
glass_0
glass_0
cup_0
glass_0
glass_0
glass_0
cup_0
glass_0
cup_0
cup_0
cup_0
cup_0
glass_0
glass_0
glass_0
cup_0
cup_0
glass_0
cup_0
glass_0
glass_0
glass_0
glass_0
glass_0
cup_0
cup_0
cup_0
cup_0
cup_0
glass_0
cup_0
glass_0
glass_0
cup_0
cup_0
glass_0
cup_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
cup_0
cup_0
cup_0
glass_0
cup_0
cup_0
cup_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
cup_0
cup_0
glass_0
glass_0
cup_0
cup_0
glass_0
glass_0
glass_0
glass_0
cup_0
glass_0
cup_0
glass_0
cup_0
glass_0
glass_0
glass_0
cup_0
glass_0
cup_0
cup_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
cup_0
glass_0
cup_0
cup_0
cup_0
glass_0
cup_0
glass_0
glass_0
glass_0
glass_0
cup_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
cup_0
glass_0
glass_0
cup_0
cup_0
cup_0
cup_0
cup_0
cup_0
cup_0
glass_0
cup_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
cup_0
cup_0
glass_0
cup_0
glass_0
glass_0
glass_0


cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
glass_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glas

glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
g

glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
glass_0
gla

glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
g

cup_30
cup_30
glass_0
cup_30
cup_100
glass_0
cup_30
cup_100
cup_30
cup_30
cup_100
glass_0
glass_0
cup_30
cup_30
glass_0
glass_0
cup_100
cup_100
cup_30
cup_30
cup_30
cup_30
cup_100
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
glass_0
cup_30
cup_30
cup_30
cup_30
cup_100
glass_0
cup_100
glass_0
cup_30
cup_30
glass_0
cup_30
cup_30
cup_30
glass_0
glass_0
cup_100
cup_30
cup_30
cup_100
cup_30
cup_100
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
cup_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30
glass_30

Successfully created the TFRecords: test.tfrecord


In [70]:
# Create Class Pbtxt
classes = ("glass_0", "glass_30", "glass_100", "cup_0", "cup_30", "cup_100")
def create_class_pbtxt(class_names, output_path):
    end = '\n'
    s = ' '
    for idx, cls in enumerate(class_names):
        out = ''
        out += 'item' + s + '{' + end
        out += s*2 + 'name:' + ' ' + '\"' + cls + '\"' + end
        out += s*2 + 'id:' + ' ' + (str(idx+1)) + end
        out += s*2 + 'display_name:' + ' ' + '\"' + cls + '\"' + end
        out += '}' + end
        with open(output_path, 'a') as f:
            f.write(out)
    print("pbtx file created successfully")
        
create_class_pbtxt(classes,'class.pbtxt')

pbtx file created successfully


In [ ]:
## What to do next? (on SSD Train)
# Train
# Test
# Convert Model to Tensorflow JS